## Setup and Configuration

In [86]:
# Import required libraries
import subprocess
import sys
import time
import requests
import json
from IPython.display import Markdown, display

# Check if the API server is running
def check_api_status():
    try:
        response = requests.get("http://localhost:8000/health", timeout=5)
        if response.status_code == 200:
            print("✅ API server is running and healthy")
            return True
        else:
            print(f"❌ API server responded with status: {response.status_code}")
            return False
    except requests.exceptions.RequestException:
        print("❌ API server is not running")
        print("💡 Please run 'python agents_custom.py' in a terminal to start the server")
        return False

# Check API status
api_running = check_api_status()

✅ API server is running and healthy


## Agent Crew Information

In [87]:
if api_running:
    # Get crew information
    response = requests.get("http://localhost:8000/crew-info")
    crew_info = response.json()
    
    print("🤖 Agent Crew Composition:")
    print("=" * 50)
    
    for agent_name, agent_info in crew_info['crew_composition'].items():
        print(f"\n👤 {agent_name.replace('_', ' ').title()}")
        print(f"   Role: {agent_info['role']}")
        print(f"   Goal: {agent_info['goal']}")
        print(f"   Tools: {', '.join(agent_info['tools'])}")
    
    print(f"\n🔄 Workflow Process:")
    for i, step in enumerate(crew_info['workflow'], 1):
        print(f"   {i}. {step}")
else:
    print("⚠️ Cannot display crew info - API server is not running")

🤖 Agent Crew Composition:

👤 Business Analyst
   Role: Senior Business Analyst
   Goal: Analyze business ideas and conduct comprehensive market research to identify opportunities and validate concepts
   Tools: Market Research Tool

👤 Strategy Consultant
   Role: Strategic Business Consultant
   Goal: Develop comprehensive business models and strategic frameworks based on market research
   Tools: Business Model Generator

👤 Product Manager
   Role: Senior Product Manager and Technical Writer
   Goal: Generate complete, detailed PRD documents by filling out templates with comprehensive content based on research and business models
   Tools: PRD Validator

🔄 Workflow Process:
   1. 1. Market Research & Analysis
   2. 2. Business Model Development
   3. 3. PRD Document Creation & Validation


## Interactive Business Idea Input

## Enhanced Input Model

The API now supports an optional `product_name` field in addition to the business idea. This allows users to:

- **Specify a custom product name** - Use your own branding and naming
- **Auto-generate from business idea** - Let the AI extract a meaningful name
- **Control PRD branding** - Ensure consistency with your product vision

### Input JSON Structure:
```json
{
  "business_idea": "Your business concept description",
  "product_name": "Optional custom product name",
  "target_market": "Optional target market description", 
  "budget_range": "Optional budget range",
  "timeline": "Optional development timeline"
}
```

In [88]:
# Define some example business ideas
example_ideas = {
    "1": {
        "business_idea": "A mobile app that uses AI to help people learn new languages through real-world conversations with native speakers",
        "product_name": "LinguaConnect AI",
        "target_market": "Language learners aged 18-35 who want practical conversation practice",
        "budget_range": "$50,000 - $100,000",
        "timeline": "12 months"
    },
    "2": {
        "business_idea": "A sustainable food delivery service that connects local organic farms directly with consumers",
        "product_name": "FarmDirect Platform",
        "target_market": "Health-conscious urban professionals and families",
        "budget_range": "$200,000 - $500,000",
        "timeline": "18 months"
    },
    "3": {
        "business_idea": "An AI-powered personal finance assistant that automatically categorizes expenses and provides personalized saving recommendations",
        "product_name": "SmartFinance AI",
        "target_market": "Young professionals and families who want better financial management",
        "budget_range": "$100,000 - $300,000",
        "timeline": "15 months"
    }
}

print("📝 Choose a business idea to transform into a PRD:")
print("=" * 60)

for key, idea in example_ideas.items():
    print(f"\n{key}. {idea['business_idea'][:80]}...")
    print(f"   Product Name: {idea['product_name']}")
    print(f"   Target: {idea['target_market']}")
    print(f"   Budget: {idea['budget_range']}")
    print(f"   Timeline: {idea['timeline']}")

print("\n4. Custom idea (enter your own)")

# Let user select an idea
choice = input("\nEnter your choice (1-4): ").strip()

if choice in example_ideas:
    selected_idea = example_ideas[choice]
    print(f"\n✅ Selected: {selected_idea['business_idea'][:100]}...")
    print(f"📦 Product Name: {selected_idea['product_name']}")
elif choice == "4":
    print("\n📝 Enter your custom business idea:")
    selected_idea = {
        "business_idea": input("Business idea: ").strip(),
        "product_name": input("Product name (optional): ").strip() or None,
        "target_market": input("Target market (optional): ").strip() or None,
        "budget_range": input("Budget range (optional): ").strip() or None,
        "timeline": input("Timeline (optional): ").strip() or None
    }
    print(f"\n✅ Custom idea: {selected_idea['business_idea'][:100]}...")
    if selected_idea['product_name']:
        print(f"📦 Product Name: {selected_idea['product_name']}")
else:
    print("❌ Invalid choice, using default idea")
    selected_idea = example_ideas["1"]

📝 Choose a business idea to transform into a PRD:

1. A mobile app that uses AI to help people learn new languages through real-world ...
   Product Name: LinguaConnect AI
   Target: Language learners aged 18-35 who want practical conversation practice
   Budget: $50,000 - $100,000
   Timeline: 12 months

2. A sustainable food delivery service that connects local organic farms directly w...
   Product Name: FarmDirect Platform
   Target: Health-conscious urban professionals and families
   Budget: $200,000 - $500,000
   Timeline: 18 months

3. An AI-powered personal finance assistant that automatically categorizes expenses...
   Product Name: SmartFinance AI
   Target: Young professionals and families who want better financial management
   Budget: $100,000 - $300,000
   Timeline: 15 months

4. Custom idea (enter your own)

✅ Selected: A mobile app that uses AI to help people learn new languages through real-world conversations with n...
📦 Product Name: LinguaConnect AI


## Generate PRD Using the Agent Crew

In [89]:
if api_running:
    print("🚀 Generating PRD using the agent crew...")
    print("⏳ This may take 2-5 minutes as the agents collaborate...")
    print("\n📊 Progress:")
    
    # Show the input being processed
    print(f"   📝 Business Idea: {selected_idea['business_idea']}")
    if selected_idea.get('product_name'):
        print(f"   📦 Product Name: {selected_idea['product_name']}")
    if selected_idea.get('target_market'):
        print(f"   🎯 Target Market: {selected_idea['target_market']}")
    if selected_idea.get('budget_range'):
        print(f"   💰 Budget Range: {selected_idea['budget_range']}")
    if selected_idea.get('timeline'):
        print(f"   ⏱️ Timeline: {selected_idea['timeline']}")
    
    # Remove None values from the request
    request_data = {k: v for k, v in selected_idea.items() if v is not None}
    
    print(f"\n📋 Request payload:")
    print(json.dumps(request_data, indent=2))
    
    # Make the API request
    start_time = time.time()
    try:
        response = requests.post(
            "http://localhost:8000/generate-prd",
            json=request_data,
            timeout=300  # 5 minutes timeout
        )
        
        end_time = time.time()
        processing_time = end_time - start_time
        
        if response.status_code == 200:
            result = response.json()
            print(f"\n✅ PRD Generated Successfully! (took {processing_time:.1f} seconds)")
            
            # Store the result for later use
            prd_result = result
            
        else:
            print(f"\n❌ Error generating PRD: {response.status_code}")
            print(f"Error details: {response.text}")
            prd_result = None
            
    except requests.exceptions.Timeout:
        print("\n⏰ Request timed out. The agent crew might need more time for complex ideas.")
        prd_result = None
    except requests.exceptions.RequestException as e:
        print(f"\n❌ Request failed: {e}")
        prd_result = None
        
else:
    print("⚠️ Cannot generate PRD - API server is not running")
    prd_result = None

🚀 Generating PRD using the agent crew...
⏳ This may take 2-5 minutes as the agents collaborate...

📊 Progress:
   📝 Business Idea: A mobile app that uses AI to help people learn new languages through real-world conversations with native speakers
   📦 Product Name: LinguaConnect AI
   🎯 Target Market: Language learners aged 18-35 who want practical conversation practice
   💰 Budget Range: $50,000 - $100,000
   ⏱️ Timeline: 12 months

📋 Request payload:
{
  "business_idea": "A mobile app that uses AI to help people learn new languages through real-world conversations with native speakers",
  "product_name": "LinguaConnect AI",
  "target_market": "Language learners aged 18-35 who want practical conversation practice",
  "budget_range": "$50,000 - $100,000",
  "timeline": "12 months"
}

✅ PRD Generated Successfully! (took 115.3 seconds)


## PRD Validation Results

In [90]:
if prd_result and prd_result.get('success', False):
    validation = prd_result.get('validation_results', {})
    
    print("📊 PRD Validation Results:")
    print("=" * 40)
    print(f"📋 Total sections required: {validation.get('total_sections', 0)}")
    print(f"✅ Sections found: {validation.get('found_sections', 0)}")
    print(f"📈 Completeness score: {validation.get('completeness_score', 0):.1f}%")
    print(f"🎯 Validation passed: {'✅ Yes' if validation.get('validation_passed', False) else '❌ No'}")
    
    missing_sections = validation.get('missing_sections', [])
    if missing_sections:
        print(f"⚠️ Missing sections: {', '.join(missing_sections)}")
    else:
        print("🎉 All required sections are present!")
        
    # Show completion bar
    score = validation.get('completeness_score', 0)
    bar_length = 30
    filled_length = int(bar_length * score / 100)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    print(f"\n📊 Completeness: |{bar}| {score:.1f}%")
else:
    print("❌ No validation results available")

📊 PRD Validation Results:
📋 Total sections required: 9
✅ Sections found: 9
📈 Completeness score: 100.0%
🎯 Validation passed: ✅ Yes
🎉 All required sections are present!

📊 Completeness: |██████████████████████████████| 100.0%


## Generated PRD Document

In [91]:
if prd_result and prd_result.get('success', False):
    prd_content = prd_result.get('prd_document', '')
    
    print("📄 Generated PRD Document:")
    print("=" * 50)
    
    # Display the PRD as markdown
    display(Markdown(prd_content))
    
    # Optionally save to file
    filename = f"generated_prd_{int(time.time())}.md"
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(prd_content)
        print(f"\n💾 PRD saved to: {filename}")
    except Exception as e:
        print(f"\n❌ Error saving PRD: {e}")
        
else:
    print("❌ No PRD content available to display")

📄 Generated PRD Document:


# Product Requirements Document: LinguaConnect AI

| Status | **Draft** |
| :--- | :--- |
| **Author** | AI Business Analyst |
| **Version** | 1.0 |

## 1. Executive Summary & Vision
LinguaConnect AI is a mobile application designed to revolutionize language learning by leveraging artificial intelligence to facilitate real-world conversations with native speakers. Our vision is to empower users to achieve fluency and confidence in new languages through practical, interactive learning experiences. By addressing the unmet need for conversational practice, LinguaConnect AI aims to capture a significant share of the growing language learning market, ultimately becoming the go-to platform for language learners worldwide.

## 2. Problem Statement & Opportunity
Language learners often struggle with applying their skills in real-life situations due to a lack of conversational practice. This gap is particularly evident among our target demographic of 18-35-year-olds, who are highly motivated to learn languages for career advancement and personal growth. The language learning market is expanding rapidly, presenting a substantial opportunity to offer a differentiated product that focuses on real-world conversation practice, leveraging AI and native speaker interactions to meet this demand.

## 3. Target Users & Personas
Our primary users are language learners aged 18-35, including:
- "Student Sam": A college student aiming to improve language skills for study abroad programs.
- "Professional Pat": A young professional seeking to enhance career opportunities through language proficiency.
- "Traveler Taylor": An avid traveler eager to communicate effectively in various countries.
These users are tech-savvy, value interactive learning experiences, and are motivated by the desire to achieve practical language skills that can be applied in real-world scenarios.

## 4. Success Metrics & Goals
Success for LinguaConnect AI will be measured by:
- User Acquisition: Achieving 1 million active users within the first year.
- Engagement: Users completing an average of 5 conversation sessions per week.
- Retention: Maintaining a 70% monthly retention rate.
- Revenue: Generating $5 million in annual revenue through premium subscriptions.
- User Satisfaction: Achieving a Net Promoter Score (NPS) of 50 or higher.

## 5. Functional Requirements & User Stories
- As a user, I want to engage in AI-driven conversation simulations, so I can practice language skills in real-world contexts.
  - Acceptance Criteria: Users can select conversation topics and receive personalized feedback.
- As a user, I want to connect with native speakers for live practice sessions, so I can improve my pronunciation and fluency.
  - Acceptance Criteria: Users can schedule sessions with native speakers and receive session summaries.
- As a user, I want to track my progress and receive personalized learning recommendations, so I can focus on areas needing improvement.
  - Acceptance Criteria: Users receive weekly progress reports and tailored learning paths.

## 6. Non-Functional Requirements (NFRs)
- Performance: The app must load within 3 seconds and handle up to 10,000 concurrent users.
- Security: User data must be encrypted and comply with GDPR regulations.
- Scalability: The system should support rapid scaling to accommodate user growth.
- Availability: The app should maintain 99.9% uptime.

## 7. Release Plan & Milestones
- Phase 1: MVP Development (Q1 2024)
  - Core AI conversation features and basic user interface.
- Phase 2: Beta Launch (Q2 2024)
  - Integration of native speaker sessions and user feedback collection.
- Phase 3: Official Launch (Q3 2024)
  - Full feature set with marketing campaign kickoff.
- Phase 4: Post-Launch Enhancements (Q4 2024)
  - Continuous improvements and additional language support.

## 8. Out of Scope & Future Considerations
- Out of Scope: Offline functionality and support for languages with limited native speakers.
- Future Considerations: Expansion into corporate language training and integration with virtual reality for immersive experiences.

## 9. Appendix & Open Questions
- Dependencies: Partnerships with native speakers and AI technology providers.
- Assumptions: Users have access to smartphones and internet connectivity.
- Open Questions: What additional languages should be prioritized for future development?
```


💾 PRD saved to: generated_prd_1761855564.md


## Summary and Next Steps

### 🎯 What We Accomplished

1. **Multi-Agent Collaboration**: Three specialized agents worked together to transform a simple business idea into a comprehensive PRD
2. **Complex Problem Solving**: Demonstrated how to solve problems that require multiple domain expertise areas
3. **Quality Validation**: Automated validation ensures the PRD meets enterprise standards
4. **Scalable Architecture**: FastAPI deployment makes it easy to integrate into existing workflows

### 🚀 Key Benefits of This Approach

- **Specialized Expertise**: Each agent focuses on what they do best
- **Consistent Output**: Template-based generation ensures standardized PRDs
- **Scalable**: Can process multiple business ideas in parallel
- **Extensible**: Easy to add new agents, tools, or validation rules

### 🔧 Possible Extensions

1. **Add Real Market Data**: Integrate with actual market research APIs
2. **Competitive Analysis**: Add an agent that researches competitors
3. **Financial Modeling**: Include detailed financial projections
4. **Risk Assessment**: Add a risk analysis agent
5. **Technical Architecture**: Include a technical architect agent for technical requirements

### 📚 Learn More

- [CrewAI Documentation](https://docs.crewai.com/)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [LangChain Documentation](https://python.langchain.com/)

---

**🎉 Congratulations! You've successfully created and tested a custom CrewAI agent system that solves a complex, multi-step business problem.**